In [1]:
import pandas as pd

## Preprocessing 

Here we'll process the dataset provided by John's Hopkins University in the public Google Sheet: https://docs.google.com/spreadsheets/d/1avGWWl1J19O_Zm0NGTGy2E-fOG05i4ljRfjl87P7FiA

The CSV export link: https://docs.google.com/spreadsheets/d/1avGWWl1J19O_Zm0NGTGy2E-fOG05i4ljRfjl87P7FiA/export?format=csv

The document has been exported as a CSV and is available in `data/COVID-19.csv`.

The processed dataset will be saved to `data/COVID-19-Cleaned.csv`.

Also, when doing analysis it'll be useful to refrence the populations of countries. This is important when trying to model Logistic Growth. We can get this data as a CSV export from the world bank at the following link: http://api.worldbank.org/countries/all/indicators/SP.POP.TOTL?format=csv

We'll download the world-bank dataset automatically and save it to `data/populations.csv`

TODO: Would be good to automatically read the document from Google Docs so that this can be re-run without a manual export step


### Covid 19 Data

In [2]:
data = pd.read_csv('data/COVID-19.csv')

data.head()

,Date,Country_Region,Province_State,Case_Type,Cases,Difference,Prep_Flow_Runtime,Latest_Date,Lat,Long
0,3/11/2020,US,Arizona,Recovered,0,0,3/23/2020 8:00:09 AM,3/22/2020,32.0575,-111.6661
1,3/11/2020,US,Arizona,Deaths,0,0,3/23/2020 8:00:09 AM,3/22/2020,32.0575,-111.6661
2,3/11/2020,US,Arizona,Confirmed,0,0,3/23/2020 8:00:09 AM,3/22/2020,32.0575,-111.6661
3,3/11/2020,US,Arizona,Active,0,0,3/23/2020 8:00:09 AM,3/22/2020,32.0575,-111.6661
4,2/15/2020,US,Arizona,Recovered,0,0,3/23/2020 8:00:09 AM,3/22/2020,32.8162,-111.2845


This dataset is represented in a "tall" or "melted" format. We'll convert it into a "wide" format along `Case_Type`. 

For the purpose of analysis we will also normalise dates from UTC format to a Y-M-D format, we will also omit `Province/State`, `Lat`, `Long`, and `Last_Update_Date` and standardize the column names into lower-case for easy manipulation in further analysis. 

#### Drop Unneeded Columns

In [3]:
clean_data = data.drop(['Province_State', 'Lat', 'Long', 'Prep_Flow_Runtime', 'Latest_Date'], axis=1)

clean_data.head()

,Date,Country_Region,Case_Type,Cases,Difference
0,3/11/2020,US,Recovered,0,0
1,3/11/2020,US,Deaths,0,0
2,3/11/2020,US,Confirmed,0,0
3,3/11/2020,US,Active,0,0
4,2/15/2020,US,Recovered,0,0


#### Rename Columns

In [4]:
clean_data = clean_data.rename(columns={
        'Country_Region': 'region',
        'Case_Type': 'case_type',
        'Date': 'date',
        'Cases': 'cumulative',
        'Difference': 'cases'
    })

clean_data.head()

,date,region,case_type,cumulative,cases
0,3/11/2020,US,Recovered,0,0
1,3/11/2020,US,Deaths,0,0
2,3/11/2020,US,Confirmed,0,0
3,3/11/2020,US,Active,0,0
4,2/15/2020,US,Recovered,0,0


#### Lower-case `case_type` values

In [5]:
clean_data['case_type'] = clean_data['case_type'].apply(str.lower)

clean_data.head()

,date,region,case_type,cumulative,cases
0,3/11/2020,US,recovered,0,0
1,3/11/2020,US,deaths,0,0
2,3/11/2020,US,confirmed,0,0
3,3/11/2020,US,active,0,0
4,2/15/2020,US,recovered,0,0


#### Widen Dataset by `case_type`

Since our granularity of our original data is uniquely identified by (region, province, date), we must sum along (region, date) to include the total count within a region of interest

In [6]:
clean_data = clean_data.pivot_table(
    index=['region', 'date'], 
    columns='case_type', 
    values=['cumulative', 'cases'],
    aggfunc='sum'
).reset_index()

clean_data.head()

region       date cumulative                             \
case_type                             active confirmed deaths recovered   
0          Afghanistan  1/23/2020          0         0      0         0   
1          Afghanistan  1/24/2020          0         0      0         0   
2          Afghanistan  1/25/2020          0         0      0         0   
3          Afghanistan  1/26/2020          0         0      0         0   
4          Afghanistan  1/27/2020          0         0      0         0   

           cases                             
case_type active confirmed deaths recovered  
0              0         0      0         0  
1              0         0      0         0  
2              0         0      0         0  
3              0         0      0         0  
4              0         0      0         0

#### Flatten Column Names

Pandas created a multi-index since we used multiple columns for our values when running the pivot operation. This will make further analysis a bit tedious. We'll flatten out the column names so that the dataset can be referenced by single column names. 

In [7]:
def process_column_name(column_tuple):
    if not column_tuple[1]:
        new_name = column_tuple[0]
    else:
        new_name = '_'.join(column_tuple)
    
    return new_name

clean_data.columns = [process_column_name(t) for t in clean_data.columns.values]

clean_data.head()

,region,date,cumulative_active,cumulative_confirmed,cumulative_deaths,cumulative_recovered,cases_active,cases_confirmed,cases_deaths,cases_recovered
0,Afghanistan,1/23/2020,0,0,0,0,0,0,0,0
1,Afghanistan,1/24/2020,0,0,0,0,0,0,0,0
2,Afghanistan,1/25/2020,0,0,0,0,0,0,0,0
3,Afghanistan,1/26/2020,0,0,0,0,0,0,0,0
4,Afghanistan,1/27/2020,0,0,0,0,0,0,0,0


#### Standardize Dates to YYYY-MM-DD Format

In [8]:
clean_data['date'] = pd.to_datetime(clean_data['date']).dt.date

clean_data.head()

,region,date,cumulative_active,cumulative_confirmed,cumulative_deaths,cumulative_recovered,cases_active,cases_confirmed,cases_deaths,cases_recovered
0,Afghanistan,2020-01-23,0,0,0,0,0,0,0,0
1,Afghanistan,2020-01-24,0,0,0,0,0,0,0,0
2,Afghanistan,2020-01-25,0,0,0,0,0,0,0,0
3,Afghanistan,2020-01-26,0,0,0,0,0,0,0,0
4,Afghanistan,2020-01-27,0,0,0,0,0,0,0,0


#### Save Dataset

Lets save this dataset so it can be used in downstream analysis

In [9]:
clean_data.sort_values(by=['region', 'date'])

,region,date,cumulative_active,cumulative_confirmed,cumulative_deaths,cumulative_recovered,cases_active,cases_confirmed,cases_deaths,cases_recovered
0,Afghanistan,2020-01-23,0,0,0,0,0,0,0,0
1,Afghanistan,2020-01-24,0,0,0,0,0,0,0,0
2,Afghanistan,2020-01-25,0,0,0,0,0,0,0,0
3,Afghanistan,2020-01-26,0,0,0,0,0,0,0,0
4,Afghanistan,2020-01-27,0,0,0,0,0,0,0,0
5,Afghanistan,2020-01-28,0,0,0,0,0,0,0,0
6,Afghanistan,2020-01-29,0,0,0,0,0,0,0,0
7,Afghanistan,2020-01-30,0,0,0,0,0,0,0,0
8,Afghanistan,2020-01-31,0,0,0,0,0,0,0,0
9,Afghanistan,2020-02-01,0,0,0,0,0,0,0,0


In [10]:
clean_data.to_csv('data/COVID-19-Cleaned.csv', index=False)

Assuming all went well, we should see a file named `COVID-19-Cleaned.csv` in our `data` directory

In [11]:
!ls data

COVID-19-Cleaned-with-Population.csv COVID-19.csv
COVID-19-Cleaned.csv                 populations.csv


## World Bank Data

We'll download the CSV from the world bank URL mentioned at the top of the document. We'll perform some basic cleanup (standardizing column names) and extract the most recent date (2018) for global populations. 

There'll be some mis-matches between country names in our two datasets, so we'll also try to match up country names across datasets.

In [12]:
url = 'http://api.worldbank.org/countries/all/indicators/SP.POP.TOTL?format=csv'

world_bank_population_data = pd.read_csv(url)

world_bank_population_data.head()

,"﻿""Country Name""",Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 62
0,Arab World,ARB,9.219775e+07,9.472451e+07,9.733444e+07,1.000342e+08,1.028328e+08,1.057364e+08,1.087586e+08,1.118994e+08,...,3.631587e+08,3.714435e+08,3.797057e+08,3.879077e+08,3.960283e+08,4.040244e+08,4.118990e+08,4.197906e+08,NaN,NaN
1,Caribbean small states,CSS,4.194710e+06,4.274060e+06,4.353628e+06,4.432217e+06,4.508198e+06,4.580374e+06,4.648367e+06,4.712526e+06,...,7.022387e+06,7.072665e+06,7.123332e+06,7.173435e+06,7.222212e+06,7.269386e+06,7.314990e+06,7.358965e+06,NaN,NaN
2,Central Europe and the Baltics,CEB,9.140176e+07,9.223274e+07,9.300950e+07,9.384002e+07,9.471580e+07,9.544099e+07,9.614634e+07,9.704327e+07,...,1.041740e+08,1.039353e+08,1.037137e+08,1.034962e+08,1.032578e+08,1.029943e+08,1.027389e+08,1.025119e+08,NaN,NaN
3,Early-demographic dividend,EAR,9.800853e+08,1.003279e+09,1.027290e+09,1.052060e+09,1.077621e+09,1.103955e+09,1.131050e+09,1.158974e+09,...,2.951856e+09,2.994853e+09,3.037663e+09,3.080325e+09,3.122842e+09,3.165142e+09,3.207189e+09,3.249141e+09,NaN,NaN
4,East Asia & Pacific,EAS,1.040958e+09,1.044545e+09,1.059019e+09,1.084796e+09,1.110214e+09,1.136691e+09,1.166600e+09,1.195270e+09,...,2.221673e+09,2.236819e+09,2.252047e+09,2.267482e+09,2.282856e+09,2.298514e+09,2.314202e+09,2.328221e+09,NaN,NaN


We'll create an initial 2-column dataset with country name and the most-recent population data from 2018. 

In [13]:
clean_pop_data = pd.DataFrame({
        'region': world_bank_population_data.iloc[:, 0],
        'population': world_bank_population_data['2018']
    })

clean_pop_data

,population,region
0,4.197906e+08,Arab World
1,7.358965e+06,Caribbean small states
2,1.025119e+08,Central Europe and the Baltics
3,3.249141e+09,Early-demographic dividend
4,2.328221e+09,East Asia & Pacific
5,2.081652e+09,East Asia & Pacific (excluding high income)
6,2.056064e+09,East Asia & Pacific (IDA & IBRD countries)
7,3.417832e+08,Euro area
8,9.187936e+08,Europe & Central Asia
9,4.177973e+08,Europe & Central Asia (excluding high income)


#### Matching Country Names

Since both datasets may have different region names refering to the same region, it'll be good to match these up and resolve any mis-match errors. 

The primary set of region names will be from the COVID-19 dataset. We'll find any missing countries from the World Bank dataset and repair errors if possible.

In [14]:
covid_regions = set(clean_data['region'].unique())
pop_regions = set(clean_pop_data['region'].unique())

These are the regions in the covid dataset that do not appear in the world bank dataset

In [15]:
covid_regions - pop_regions

{'Brunei',
 'Cape Verde',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Cruise Ships',
 'Czechia',
 'East Timor',
 'Egypt',
 'Holy See',
 'Iran',
 'Korea, South',
 'Kyrgyzstan',
 'Martinique',
 'Russia',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Slovakia',
 'Syria',
 'Taiwan*',
 'US',
 'Venezuela'}

These are the regions in the world bank dataset that do not appear in our COVID-19 dataset

In [16]:
pop_regions - covid_regions

{'American Samoa',
 'Arab World',
 'Aruba',
 'Belize',
 'Bermuda',
 'Botswana',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Burundi',
 'Caribbean small states',
 'Cayman Islands',
 'Central Europe and the Baltics',
 'Channel Islands',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Curacao',
 'Czech Republic',
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East Asia & Pacific (IDA & IBRD countries)',
 'East Asia & Pacific (excluding high income)',
 'Egypt, Arab Rep.',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Europe & Central Asia (excluding high income)',
 'European Union',
 'Faroe Islands',
 'Fragile and conflict affected situations',
 'French Polynesia',
 'Gibraltar',
 'Greenland',
 'Guam',
 'Guinea-Bissau',
 'Heavily indebted poor countries (HIPC)',
 'High income',
 'Hong Kong SAR, China',
 'IBRD only',
 'IDA & IBRD total',
 'IDA blend',
 'IDA only',
 'IDA total',
 'Iran, Islamic Rep.',
 'Isle of Man',
 'Kiribati'

From manual inspection, we can see mismatches occur in the COVID-19 dataset due to the use of acronyms (US instead of United States), short-form representations, and some omissions (Taiwan, Palestine, Vatican City and others don't appear in the World Bank Dataset). 

We will fix any mis-matches we can. We'll omit regions from the world bank dataset that may appear in the COVID-19 dataset, just be aware that there will be some missing data if doing any joins between the two datasets. 

We'll translate the names so they are standardized to the COVID-19 dataset. 

In [17]:
translation_table = {
    'United States': 'US',
    'Brunei Darussalam': 'Brunei',
    'Egypt, Arab Rep.': 'Egypt',
    'Hong Kong SAR, China': 'Hong Kong',
    'Iran, Islamic Rep.': 'Iran',
    'Macao SAR, China': 'Macau',
    'Russian Federation': 'Russia',
    'Slovak Republic': 'Slovakia',
    'Korea, Rep.': 'Korea, South',
    'St. Martin (French part)': 'St. Martin',
    'Congo, Dem. Rep.': 'Congo (Kinshasa)'
}

def rename_region(name):
    try: 
        return translation_table[name]
    except KeyError:
        return name

In [18]:
clean_pop_data['region'] = clean_pop_data['region'].apply(rename_region)

#### Regions in COVID-19 Dataset Omitted from World Bank Dataset

These are the regions that did not appear in our World Bank dataset

In [19]:
pop_regions2 = set(clean_pop_data['region'].unique())

covid_regions - pop_regions2

{'Cape Verde',
 'Congo (Brazzaville)',
 'Cruise Ships',
 'Czechia',
 'East Timor',
 'Holy See',
 'Kyrgyzstan',
 'Martinique',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Syria',
 'Taiwan*',
 'Venezuela'}

#### Drop Regions from World Bank Dataset

Lets drop any regions in the World Bank Dataset that do not appear in the COVID-19 Dataset

In [20]:
clean_pop_data = clean_pop_data.loc[
    clean_pop_data['region'].apply(lambda x: x in covid_regions)
]

#### Save Population Dataset 

In [21]:
clean_pop_data.to_csv('data/populations.csv', index=False, columns=['region', 'population'])

Assuming all went well, we should see a file named `populations.csv` in our data directory

In [22]:
!ls data

COVID-19-Cleaned-with-Population.csv COVID-19.csv
COVID-19-Cleaned.csv                 populations.csv


#### Attach Population Data to COVID-19 Dataset 
For convenience, we'll attach the population data to our COVID-19 dataset so we don't need to do this step during analysis. We'll save this to `data/COVID-19-Cleaned-with-Population.csv`


In [23]:
clean_data_with_population = clean_data.merge(clean_pop_data, on='region', how='left')

clean_data_with_population.head()

,region,date,cumulative_active,cumulative_confirmed,cumulative_deaths,cumulative_recovered,cases_active,cases_confirmed,cases_deaths,cases_recovered,population
0,Afghanistan,2020-01-23,0,0,0,0,0,0,0,0,37172386.0
1,Afghanistan,2020-01-24,0,0,0,0,0,0,0,0,37172386.0
2,Afghanistan,2020-01-25,0,0,0,0,0,0,0,0,37172386.0
3,Afghanistan,2020-01-26,0,0,0,0,0,0,0,0,37172386.0
4,Afghanistan,2020-01-27,0,0,0,0,0,0,0,0,37172386.0


#### Save Combined Dataset 

In [24]:
clean_data_with_population.to_csv('data/COVID-19-Cleaned-with-Population.csv', index=False)

In [25]:
!ls data

COVID-19-Cleaned-with-Population.csv COVID-19.csv
COVID-19-Cleaned.csv                 populations.csv
